 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="#What-is-Random-Forest" data-toc-modified-id="What-is-Random-Forest-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>What is Random Forest</a></span></li><li><span><a href="#The-algorithm" data-toc-modified-id="The-algorithm-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>The algorithm</a></span></li><li><span><a href="#Data-Prep" data-toc-modified-id="Data-Prep-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Prep</a></span></li><li><span><a href="#Decision-Tree-and-Random-Forest" data-toc-modified-id="Decision-Tree-and-Random-Forest-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Decision Tree and Random Forest</a></span><ul class="toc-item"><li><span><a href="#Decision-tree-basics" data-toc-modified-id="Decision-tree-basics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Decision tree basics</a></span></li><li><span><a href="#Implementation-of-Tree" data-toc-modified-id="Implementation-of-Tree-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Implementation of Tree</a></span></li></ul></li></ul></div>

## What is Random Forest

>Decision trees can suffer from high variance which makes their results fragile to the specific training data used.

>Building multiple models from samples of your training data, called bagging, can reduce this variance, but the trees are highly correlated.

>Random Forest is an extension of bagging that in addition to building trees based on multiple samples of your training data, it also constrains the features that can be used to build the trees, forcing trees to be different. This, in turn, can give a lift in performance.

## The algorithm

>Decision trees involve the greedy selection of the best split point from the dataset at each step.

>This algorithm makes decision trees susceptible to high variance if they are not pruned. This high variance can be harnessed and reduced by creating multiple trees with different samples of the training dataset (different views of the problem) and combining their predictions. This approach is called bootstrap aggregation or bagging for short.

>A limitation of bagging is that the same greedy algorithm is used to create each tree, meaning that it is likely that the same or very similar split points will be chosen in each tree making the different trees very similar (trees will be correlated). This, in turn, makes their predictions similar, mitigating the variance originally sought.

>We can force the decision trees to be different by limiting the features (rows) that the greedy algorithm can evaluate at each split point when creating the tree. This is called the Random Forest algorithm.

>Like bagging, multiple samples of the training dataset are taken and a different tree trained on each. The difference is that at each point a split is made in the data and added to the tree, only a fixed subset of attributes can be considered.

>For classification problems,  the number of attributes to be considered for the split is limited to the square root of the number of input features.

>The result of this one small change are trees that are more different from each other (uncorrelated) resulting predictions that are more diverse and a combined prediction that often has better performance that single tree or bagging alone.

## Data Prep

Sample data used is the sonar dataset.

In [4]:
%mkdir -p data/research

In [3]:
import urllib.request as request
file_path = 'data/research/sonar.all-data.csv'
d_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data'
request.urlretrieve(d_url, file_path)

('data/research/sonar.all-data.csv', <http.client.HTTPMessage at 0x108f47128>)

In [4]:
import pandas as pd
df = pd.read_csv(file_path, header=None)

In [5]:
df.shape

(208, 61)

## Decision Tree and Random Forest

### Decision tree basics

In a decision tree, split points are chosed by finding the feature and the value of that feature which results in lowerst cost.

For classification problem, this cost is usually evaluated by a cost function called Gini index. Gini index calculates the purity of the group of data created by the split point.

A tree node is pure (`gini = 0`) if all instances it applies to belong to the same class.

*Gini Impurity* is measured as 
$$
G_i = 1 - \sum_{k=1}^n p_{i,k}^2
$$

where $p_{i,k}$ refers to the ratio of class $k$  instances among the whole input instances in the $i^{th}$ node.

For example, assume there is a node with 54 input instances, 0 of them belong to class A, 49 of them belong to class B, and 5 of them belong to class C. Then the gini score is $1 - (0/54)^2 - (49/54)^2 - (5/54)^2 \approx 0.168$

In our case, we only want a binary classifier outputing `relevant (1)` or `irrelavent (0)`. So if a node perfectly separated the input into one class(leaf), the *gini impurity* will be 0.

Another measure will be *Entropy*:
$$H_i = - \sum_{k=1 \mid p_{i,k} \neq 0}^n p_{i,k}log(p_{i,k})$$
Note *Entropy* is more expensive as it uses $log$.

> - Gini is intended for continuous attributes, and Entropy for attributes that occur in classes
- Gini is to minimize misclassification
- Entropy is for exploratory analysis
- Entropy may be a little slower to compute

General Implementation of both:

In [8]:
def calc_shannon_entropy(self, left, right):
        left_sum = sum(left.values())
        right_sum = sum(right.values())
        if 0 in left.values():
            left_entropy = 0
        else:
            left_entropy = sum([-(i/left_sum)*np.log2(i/left_sum) for i in left.values()])

        if 0 in right.values():
            right_entropy = 0
        else:
            right_entropy = sum([-(i/right_sum)*np.log2(i/right_sum) for i in right.values()])
        entropy = (left_entropy*left_sum + right_entropy*right_sum) / (left_sum + right_sum)
        return entropy

In [9]:
def cal_gini_index(data):
    pass

### Implementation of Tree

In [12]:
import math
import numpy as np
import random

In [121]:
f = 5
label_ind=60
x = 3
print(df[f][x], df[label_ind][x])
print(df)

0.0368
         0       1       2       3       4       5       6       7       8   \
0    0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1    0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2    0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3    0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4    0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   
5    0.0286  0.0453  0.0277  0.0174  0.0384  0.0990  0.1201  0.1833  0.2105   
6    0.0317  0.0956  0.1321  0.1408  0.1674  0.1710  0.0731  0.1401  0.2083   
7    0.0519  0.0548  0.0842  0.0319  0.1158  0.0922  0.1027  0.0613  0.1465   
8    0.0223  0.0375  0.0484  0.0475  0.0647  0.0591  0.0753  0.0098  0.0684   
9    0.0164  0.0173  0.0347  0.0070  0.0187  0.0671  0.1056  0.0697  0.0962   
10   0.0039  0.0063  0.0152  0.0336  0.0310  0.0284  0.0396  0.0272  0.0323   
11   0.0123  0.0309  0.0169  0.0313  0.0358  

In [6]:
'''
A dummy version of tree nodes
'''
class Node:
      
    def __init__(self, data, rows, features, depth, max_depth):
        self.left = None
        self.right = None
        self.data = data
        self.rows = rows
        self.features = features
        self.label_index = 60
        self.labels = ['R', 'M']
        self.spliting_feature_val = None
        self.id = '%030x' % random.randrange(16**30)
        self.depth = depth
        self.max_depth = max_depth
        self.min_feature = None
        self.min_break_point = None
        self.min_gini = None

    
    def calc_shannon_entropy(self):
        raw_val = 0
        for label in self.labels:
            members = self.data.loc[self.data[self.label_index] == label]
            if len(members) <= 0: continue
            filtered = [x for x in members.index.values if x in self.rows]
            intermediate = len(filtered)/len(self.rows)
            raw_val += -intermediate*np.log2(intermediate)
        return raw_val
    
    def calc_gini_index(self):
        raw_val = 1
        members = [self.data[self.label_index][x] for x in self.rows]
        for label in self.labels:
#             members = self.data.loc[self.data[self.label_index] == label]
            #maybe do as a for loop?
            filtered = [x for x in members if x == label]
#             filtered = members
            raw_val -= (len(filtered)/len(self.rows))**2
        return raw_val
    
        
    '''
    calculate info gain from gini/entropy
    '''
    def cal_info_gain():
        pass
    
    
    def find_break_points(self, df, feature):
        breaks = []
        for i in range(len(df)-1):
            row = df[i:i+1]
            next_row = df[i+1:i+2]
#             print(row[self.label_index])
            if row[self.label_index].values[0] != next_row[self.label_index].values[0]:
                breaks.append(next_row[feature].values[0]) #float precision issue, care
        return breaks
    
        
    '''
    Choose the best feature to split at this point
    i.e. low gini/entropy, high infoGain
    '''
    
    def split(self):
        #are we a leaf node?
        if len(self.rows) == 0:
            raise ValueError('The node has no document feed, no more splitting')
        elif self.calc_gini_index() == 0:
            raise ValueError('The node is pure, no more splitting')
        elif self.depth == self.max_depth:
            raise ValueError('The node has reached max recursion depth, no more splitting')
        elif len(self.features) == 0:
            raise ValueError('There are no more features to split on.')
            
        #we are not a leaf node.
        min_gini, min_feature, min_break_point, new_left, new_right = 2, -999, -999, None, None
        bp_len_sum = 0
        for feature in self.features:
#             print('parsing')
            to_parse = [(self.data[feature][x],self.data[self.label_index][x]) for x in self.rows]
            to_parse = pd.DataFrame(to_parse, columns=(feature,self.label_index), index=self.rows)
#             print(to_parse)
            to_parse.sort_values(feature, inplace=True)
#             print(to_parse)
#             to_parse = self.data[[feature, self.label_index]]
#             to_parse = to_parse.loc[to_parse.index.isin (self.rows)]
#             to_parse.sort_values(feature, inplace=True)
#             print(to_parse)
            break_points = self.find_break_points(to_parse, feature)
#             print(break_points)
            bp_len_sum += len(break_points)
            for break_point in break_points:
                left = Node(self.data, to_parse.loc[to_parse[feature] < break_point].index.values, [x for x in self.features if x != feature], self.depth+1, self.max_depth)
#                 left = Node(to_parse.loc[to_parse[feature] <break_point]
                right = Node(self.data, to_parse.loc[to_parse[feature] >= break_point].index.values, [x for x in self.features if x != feature], self.depth+1, self.max_depth)
                ## We should ajdust this so it pass self.data and reference of rows and cols
#                 print(left.index.values)

#                 print(self.calc_gini_index(left))
                left_gini = left.calc_gini_index() if len(left.rows) > 0 else 0
                right_gini = right.calc_gini_index() if len(right.rows) > 0 else 0
                total_gini = (left_gini*len(left.rows) + right_gini*len(right.rows) )/(len(left.rows) + len(right.rows) )
#                 min_gini = min(total_gini, min_gini)
#                 print(self.id, feature, total_gini)
                if total_gini < min_gini:
                    min_gini, min_break_point, min_feature, new_left, new_right = total_gini, break_point, feature, left, right
        if bp_len_sum == 0:
            print(to_parse)
            print(self.calc_gini_index())
        self.left = new_left
        self.right = new_right
        self.min_feature, self.min_break_point, self.min_gini = min_feature, min_break_point, min_gini
        try:
            if self.left is None:
                print(self.min_feature,self.min_break_point,self.min_gini)
            self.left.split()
        except ValueError: # probably need a customized error class
            pass
        try:
            self.right.split()
        except ValueError:
            pass
        
    def __str__(self):
#         if self.left and self.right:
        children = [x.id for x in (self.left, self.right)] if self.left and self.right else []
        return "[{ID}, {Gini}, {Size}, {Feature}, {BP}, {Children}]".format(ID=self.id, 
                                                            Gini = self.calc_gini_index(),
                                                            Size = len(self.rows),
                                                            Feature=self.min_feature, 
                                                            BP=self.min_break_point,
                                                           Children=children)
#         else:
#             "[{ID}, (Children=None)]".format(ID=self.id)
    
    def get_proportions(self, target_label):
        members = [self.data[self.label_index][x] for x in self.rows]
        filtered = [x for x in members if x == target_label]
#         members = self.data.loc[self.data[self.label_index] == target_label]
#         filtered = [x for x in members.index.values if x in self.rows]
        raw_val = (len(filtered)/len(self.rows))
        return raw_val
        

In [173]:
ls = [x for x in range(60)]
dummy = Node(df, [1,2,2,7,4,5,200,150, 175, 175, 130], ls, 0, 2)
dummy.split()
# print(dummy.left.split())

         0  60
200  0.0131  M
150  0.0209  M
2    0.0262  R
5    0.0286  R
175  0.0294  M
130  0.0443  M
1    0.0453  R
7    0.0519  R
4    0.0762  R
[0.026200000000000001, 0.029399999999999999, 0.0453]
         1  60
175  0.0123  M
150  0.0278  M
200  0.0387  M
130  0.0446  M
5    0.0453  R
1    0.0523  R
7    0.0548  R
2    0.0582  R
4    0.0666  R
[0.0453]
         2  60
150  0.0115  M
175  0.0117  M
130  0.0235  M
5    0.0277  R
200  0.0329  M
4    0.0481  R
7    0.0842  R
1    0.0843  R
2    0.1099  R
[0.027699999999999999, 0.032899999999999999, 0.048099999999999997]
         3  60
200  0.0078  M
175  0.0113  M
5    0.0174  R
7    0.0319  R
4    0.0394  R
150  0.0445  M
1    0.0689  R
130  0.1008  M
2    0.1083  R
[0.017399999999999999, 0.044499999999999998, 0.068900000000000003, 0.1008, 0.10829999999999999]
         4  60
5    0.0384  R
150  0.0427  M
175  0.0497  M
4    0.0590  R
200  0.0721  M
2    0.0974  R
7    0.1158  R
1    0.1183  R
130  0.2252  M
[0.042700000000000002, 0.

         38 60
200  0.1416  M
1    0.1840  R
130  0.2236  M
150  0.2654  M
4    0.2828  R
175  0.2863  M
5    0.4054  R
7    0.4288  R
2    0.5375  R
[0.184, 0.22359999999999999, 0.2828, 0.2863, 0.40539999999999998]
         39 60
130  0.1180  M
200  0.1460  M
150  0.1760  M
1    0.1970  R
4    0.2430  R
7    0.2546  R
175  0.2634  M
5    0.3296  R
2    0.4719  R
[0.19700000000000001, 0.26340000000000002, 0.3296]
         40 60
175  0.0541  M
200  0.0846  M
130  0.1103  M
7    0.1151  R
150  0.1599  M
1    0.1674  R
4    0.1979  R
5    0.2707  R
2    0.4647  R
[0.11509999999999999, 0.15989999999999999, 0.16739999999999999]
         41 60
1    0.0583  R
150  0.0866  M
200  0.1055  M
175  0.1874  M
7    0.2196  R
4    0.2444  R
2    0.2587  R
5    0.2650  R
130  0.2831  M
[0.086599999999999996, 0.21959999999999999, 0.28310000000000002]
         42 60
150  0.0590  M
5    0.0723  R
1    0.1401  R
200  0.1639  M
4    0.1847  R
7    0.1879  R
2    0.2129  R
130  0.2385  M
175  0.3459  M
[0.0

In [13]:
nodes = [dummy]
while(len(nodes) > 0):
    new_nodes = []
    level_str = ''
    for node in nodes:
        level_str += str(node) + "\n"
        if node.left:
            new_nodes.append(node.left)
        if node.right:
            new_nodes.append(node.right)
    print(level_str+"\n--------------------------------------------------", end='\n')
    nodes = new_nodes

[2ef62a70265f1d9997e4d47fca6bb8, 10, 0.1989, ['9c0dd87015a01f9f519a55a2964a6e', '0c2962436230065c9df75a6370f251']]

--------------------------------------------------
[9c0dd87015a01f9f519a55a2964a6e, 3, 0.0525, ['7d1487741d4ed031b81f842edcca4c', 'cb4473455a49e874e700b7ed1bc9b2']]
[0c2962436230065c9df75a6370f251, 15, 0.6699, ['2324225d9adafafa05568774a45ba8', '2f3e71606996cabfaa3bd4485d4848']]

--------------------------------------------------
[7d1487741d4ed031b81f842edcca4c, (Children=None)]
[cb4473455a49e874e700b7ed1bc9b2, (Children=None)]
[2324225d9adafafa05568774a45ba8, (Children=None)]
[2f3e71606996cabfaa3bd4485d4848, (Children=None)]

--------------------------------------------------


In [7]:
'''
A dummy implementation of decision trees
'''
class Tree:
    
    '''
    params:
    train_data - training data to trainthe tree
    depth - max recursion depth of the tree
    benchmark - benchmark for geni/entropy
    '''
    def __init__(self, train_data, depth, benchmark, rows, features): #should we include data here
        self.depth = depth
        self.rows = rows
        self.features = features
        self.data = train_data
        self.benchmark = benchmark
        self.head = Node(train_data, rows, features, 0, depth)
        
    '''
    Recursively split until geni/entropy benchmark met or max_depth reached
    '''
    def fit(self):
        #think about behavior of pure nodes more
        try:
            self.head.split()
        except ValueError: #change this to whatever node.split() throws
            print('Head is a pure node.')
    '''
    params: 
    test_data - test data to run the prediction on
    
    return: 
    outputs confidence/probability of each category
    '''
    def predict(self, test_data):
#         assuming input data is a list right now
        cur_node = self.head
        while (cur_node.left and cur_node.right):
            if (test_data[cur_node.min_feature].values[0] < cur_node.min_break_point):
                cur_node = cur_node.left
            else:
                cur_node = cur_node.right
        
#         here, cur_node should be the leaf
        r_confidence = cur_node.get_proportions('R')
        m_confidence = cur_node.get_proportions('M')
        
        return (r_confidence, m_confidence)
    
    '''
    params: 
    more_data - more training data to update the tree
    
    return: 
    Null or we can say something like which nodes are changed
    '''
    def update(more_data):
        pass
    
    '''
    Maybe we can use pickle for this
    '''
    def store_tree(file_path):
        pass
    
    def load_tree(file_path):
        pass
    
    '''
    String representation
    '''
    def __str__(self):
        string = ''
        string += str(sorted(self.features))
        string += '\n'
        nodes = [self.head]
        while(len(nodes) > 0):
            new_nodes = []
            level_str = ''
            for node in nodes:
                level_str += str(node) + "\n"
                if node.left:
                    new_nodes.append(node.left)
                if node.right:
                    new_nodes.append(node.right)
            string += level_str+"\n--------------------------------------------------\n"
            nodes = new_nodes
        return string

In [15]:

ls = [x for x in range(60)]
# dummy = Node(df, range(df.shape[0]-20), ls, 0, 2)



In [16]:
tree.predict(df[200:201])

NameError: name 'tree' is not defined

In [ ]:
df[200:201][60].values[0]

In [278]:
def cross_val(df, tries):
    for i in range(tries):
        shuffle = df.sample(frac=1)
        tree = Tree(df, 3, None, range(shuffle.shape[0]-20), ls)
        tree.fit()
        score = 0
        for i in range(188, 208):
            actual = shuffle[i:i+1][60].values[0]
            p = tree.predict(shuffle[i:i+1])
            if p[0] > p[1]:
        #         print('R/{}'.format(actual))
                if 'R' == actual: 
                    score+=1
            else:
        #         print('M/{}'.format(actual))
                if 'M' == actual: 
                    score+=1
        print(score/(208-188))

In [279]:
cross_val(df, 5)

0.95
0.95
0.95
0.95
0.85


In [8]:
class AlreadyFitException(Exception):
    pass

In [15]:
'''
Dummy Version of Random Forest
'''
class RNF: 
    '''
    params:
    train_data - training data to trainthe tree
    n_trees - number of trees to setup
    tree_depth - max recursive
    random_seed - seed for random gen
    n_max_features - max num of features to pass to each tree
    n_max_input - max num of input to pass to each tree
    '''
    def __init__(self, train_data, n_trees, tree_depth, random_seed, n_max_features, n_max_input):
        self.trees = []
        self.train_data = train_data
        self.n_trees = n_trees
        self.tree_depth = tree_depth
        self.n_max_features = n_max_features
        self.n_max_input = n_max_input
#         self.features = [()] #list of tuples like (tree, emails, features)
        random.seed(random_seed)
    
        np.random.seed(random_seed)
        pass
    
    '''
    Randomly select features and emails from the train_data 
    '''
    def random_select(self, train_data):
        selected_rows = np.random.choice(self.train_data.shape[0], self.n_max_input)
        selected_features = np.random.choice(self.train_data.shape[1] - 1, self.n_max_features, replace=False)
        return (selected_rows, selected_features)
        
    '''
    pass randomly selected emails and features to each tree
    '''
    def fit(self):
        if len(self.trees) != 0:
            raise AlreadyFitException('This forest has already been fit to the data')
        for i in range(self.n_trees):
            selected = self.random_select(self.train_data)
#             self, train_data, depth, benchmark, rows, features
            self.trees.append(Tree(self.train_data, self.tree_depth, 0, selected[0], selected[1]))
        for tree in self.trees:
            tree.fit()
    
    '''
    calculate a proba from output of each tree's prediction
    should ouput two arrays: probas and classfication
    '''
    def some_majority_count_metric(self, scores):
        return np.mean(scores, axis=0)
    
    def predict(self, test_data):
        scores = [tree.predict(test_data) for tree in self.trees]
        probas = self.some_majority_count_metric(scores)
        classes = 'R' if probas[0] > probas[1] else 'M'
        return self.some_majority_count_metric(scores), classes
    
    '''
    params: 
    more_data - more training data to update the forest
    
    return: 
    Null or we can say something like which trees are changed
    '''
    def update(more_data):
        pass
    
    '''
    Maybe we can use pickle for this
    '''
    def store_rnf(file_path):
        pass
    
    def load_rnf(file_path):
        pass

In [281]:
a = RNF(df[:188], 2, 3, 42, 40, 80)

In [282]:
a.fit()

In [261]:
for tree in a.trees:
    print(tree)

[1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 16, 18, 19, 21, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 45, 48, 49, 50, 51, 53, 54, 55, 57, 58, 59]
[477183392456de3eb13b9046685257, 0.4921875, 80, 49, 0.0183, ['ba26a744ed4fd51c2c8fb14de5f2f7', '00463e0a77b5cc2e8746510c2ca7fa']]

--------------------------------------------------
[ba26a744ed4fd51c2c8fb14de5f2f7, 0.314098750743605, 41, 18, 0.969, ['983c2e15e9bee44e1439774d126ab1', 'ec1c6da0d980baed57a5c89110f698']]
[00463e0a77b5cc2e8746510c2ca7fa, 0.4260355029585799, 39, 3, 0.0226, ['68c68e3d231e3b4a97a245da5bf081', '637f145ae57068d2ed9291c3da1521']]

--------------------------------------------------
[983c2e15e9bee44e1439774d126ab1, 0.2285318559556786, 38, 54, 0.0036, ['c6601d0aaa3c511cd97b172f51270f', '98e361bf4d493dbc57003e91f91586']]
[ec1c6da0d980baed57a5c89110f698, 0.0, 3, None, None, []]
[68c68e3d231e3b4a97a245da5bf081, 0.0, 6, None, None, []]
[637f145ae57068d2ed9291c3da1521, 0.2975206611570247, 33, 8, 0.1237, ['d579efd2d3ea0a

In [283]:
a.predict(df[200:201])

(array([ 0.83333333,  0.16666667]), 'R')

In [263]:
df[200:201][60]

200    M
Name: 60, dtype: object

In [16]:
def cross_val_rnf(df, tries):
    for i in range(tries):
        shuffle = df.sample(frac=1)
        forest = RNF(df[0:188], 50, 4, random.randint(1, 999), 40, 80)
        forest.fit()
        score = 0
        for i in range(188, 208):
            actual = shuffle[i:i+1][60].values[0]
            p = forest.predict(shuffle[i:i+1])[1]
            if p[0] > p[1]:
        #         print('R/{}'.format(actual))
                if 'R' == actual: 
                    score+=1
            else:
        #         print('M/{}'.format(actual))
                if 'M' == actual: 
                    score+=1
        print(score/(208-188))

In [17]:
cross_val_rnf(df, 10)

IndexError: string index out of range